#### Define Imports

In [1]:
#%pip install openai

hugging face - chunking strategies
embedding the chunks into vectors
chroma (vector store) 
retrieval method - API calls/
-> question -> chunks from store -> LLM -> answer question based on context

Can we have chat gpt create the API call string?
->prompt and examples

Separate prompt for Park to ParkCode?

Evaluation
RAGAS - for RAG evaluation, explore what types of evaluation, retrieval-focused
Correct generation - some manual evaluation may be necessary 

Evaluate chatGpt vs a traditional NLP key-word matching on the same queries
Set up baseline architecture for experimentation

In [1]:
import requests
import pandas as pd 
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import openai
import os
from openai import OpenAI
import time
import sys

sys.path.insert(0,'../')
from environment import env

#### Define Variables

In [3]:
config = env.env()
nps_api_key = config['nps_api_key']
openai.api_key = config['gpt_api_key']
gpt_npi_key = config['gpt_api_key']
client = OpenAI(api_key  = config['gpt_api_key'])
api_base_url = 'https://developer.nps.gov/api/v1/'

#### Define Functions

In [23]:
def handle_query(query, model):
    prompt = f"prompt: {query}\n"
    response = client.completions.create(
        model=model,  
        prompt=prompt,
        max_tokens=50
    )
    completion = response.choices[0].text#.strip()
    return completion

#### Read Inputs

In [6]:
synthetic_queries_df = pd.read_csv('synthetic_queries.csv')
train_file =  client.files.create(
  file=open("fine_tune_train_data.jsonl", "rb"),
  purpose="fine-tune"
)

val_file = client.files.create(
  file=open("fine_tune_val_data.jsonl", "rb"),
  purpose="fine-tune"
)

train_file_id = train_file.id
val_file_id = val_file.id

#### Fine Tune

In [ ]:
fine_tune = client.fine_tuning.jobs.create(
    model="davinci-002",
    training_file=train_file_id,
    validation_file=val_file_id,
    seed = 42,
    suffix = 'nps_model'
)

In [9]:
try:
    fine_tune_id = fine_tune.id
except:
    fine_tune_id = config['gpt_model_id']

In [10]:
while True:
    status = client.fine_tuning.jobs.retrieve(fine_tune_id).status
    print(f"Fine-tuning job status: {status}")

    if status in ["succeeded", "failed"]:
        break

    time.sleep(60)

Fine-tuning job status: succeeded


#### Test Fine Tuning

In [25]:
model = client.fine_tuning.jobs.retrieve(fine_tune_id).fine_tuned_model

queries = ["Are there any alerts for the African American War Memorial","Where are the parking lots in Yosemite?","What amenities does the Jimmy Carter Park have?"]
for query in queries:
    api_call = handle_query(query,model)
    print(api_call)

alerts afamalerts afam Inalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts afamalerts
parkinglots yoseparkinglots yoseparkinglots yose.
parkinglots yoseparkinglots yose.
parkinglots yoseparkinglots yoseparkinglots yose.

parkinglots yoseparking
amenities jicaamenities jica Richard Nixonamenities rich Theamenities rich RichardParks richRichardparks rich Richardparks rich Richardparks rich Richardparks richRichardparks rich Richardparks richRichardparks rich Richard
